In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate(); Pkg.add(PackageSpec(name="Flux", rev = "master"))

using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/notebooks/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/notebooks/vision/mnist/Manifest.toml`
 [no changes]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
│   exception = ErrorException("Required dependency Juno [e5e0dc1b-0480-54bc-9374-aad01c23163d] failed to load from a cache file.")
└ @ Base loading.jl:969


using CuArrays

Classify MNIST digits with a simple multi-layer-perceptron

In [2]:
imgs = MNIST.images()

60000-element Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8

Stack images into one large batch

In [3]:
X = hcat(float.(reshape.(imgs, :))...) |> gpu

labels = MNIST.labels()

60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 ⋮
 2
 9
 5
 1
 8
 3
 5
 6
 8

One-hot-encode the labels

In [4]:
Y = onehotbatch(labels, 0:9) |> gpu

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200) |> gpu
evalcb = () -> @show(loss(X, Y))
opt = ADAM(params(m))

@info "starting training.."
Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Info: starting training..
└ @ Main.##360 string:16
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1
loss(X, Y) = 2.3894427f0 (tracked)
loss(X, Y) = 2.1580908f0 (tracked)
loss(X, Y) = 1.9806883f0 (tracked)
loss(X, Y) = 1.8056185f0 (tracked)
loss(X, Y) = 1.6421576f0 (tracked)
loss(X, Y) = 1.4961658f0 (tracked)
loss(X, Y) = 1.3667434f0 (tracked)
loss(X, Y) = 1.2496492f0 (tracked)
loss(X, Y) = 1.1401035f0 (tracked)
loss(X, Y) = 1.0379163f0 (tracked)
loss(X, Y) = 0.9453742f0 (tracked)
loss(X, Y) = 0.86320263f0 (tracked)
loss(X, Y) = 0.7916681f0 (tracked)
loss(X, Y) = 0.73016065f0 (tracked)
loss(X, Y) = 0.6777168f0 (tracked)
loss(X, Y) = 0.63317394f0 (tracked)
loss(X, Y) = 0.59533244f0 (tracked)
loss(X, Y) = 0.5631112f0 (tracked)
loss(X, Y) = 0.5354955f0 (tracked)
loss(X, Y) = 0.5116075f0 (tr

0.9219666666666667

Test set accuracy

In [5]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

accuracy(tX, tY)

0.9219